In [1]:
import pandas as pd
import os
import zipfile
import shutil
import matplotlib.pyplot as plt
import cv2

In [2]:
# ------------------- 1. Montar Google Drive -------------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# ------------------- 2. Definir rutas -------------------
# Ruta en Google Drive
proyecto_path = '/content/drive/MyDrive/ProyectoIPD441'
zip_path = f'{proyecto_path}/portadas_data_clip.zip'
csv_drive_path = f'{proyecto_path}/data_clip_limpia.csv'

# Rutas en Colab
extract_to = '/content'
csv_colab_path = '/content/data_clip_limpia.csv'

In [4]:
# ------------------- 3. Copiar CSV a /content -------------------
shutil.copy(csv_drive_path, csv_colab_path)
print("✅ CSV copiado a /content")

✅ CSV copiado a /content


In [5]:
# ------------------- 4. Descomprimir imágenes con progreso -------------------
os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    all_files = zip_ref.namelist()
    total = len(all_files)
    print(f"Total de imágenes a descomprimir: {total}")
    for i, file in enumerate(all_files, 1):
        zip_ref.extract(file, extract_to)
        if i % 1000 == 0:
            print(f"✅ {i} imágenes descomprimidas...")
    print(f"✅ Descompresión completa: {i} imágenes extraídas.")

Total de imágenes a descomprimir: 50716
✅ 1000 imágenes descomprimidas...
✅ 2000 imágenes descomprimidas...
✅ 3000 imágenes descomprimidas...
✅ 4000 imágenes descomprimidas...
✅ 5000 imágenes descomprimidas...
✅ 6000 imágenes descomprimidas...
✅ 7000 imágenes descomprimidas...
✅ 8000 imágenes descomprimidas...
✅ 9000 imágenes descomprimidas...
✅ 10000 imágenes descomprimidas...
✅ 11000 imágenes descomprimidas...
✅ 12000 imágenes descomprimidas...
✅ 13000 imágenes descomprimidas...
✅ 14000 imágenes descomprimidas...
✅ 15000 imágenes descomprimidas...
✅ 16000 imágenes descomprimidas...
✅ 17000 imágenes descomprimidas...
✅ 18000 imágenes descomprimidas...
✅ 19000 imágenes descomprimidas...
✅ 20000 imágenes descomprimidas...
✅ 21000 imágenes descomprimidas...
✅ 22000 imágenes descomprimidas...
✅ 23000 imágenes descomprimidas...
✅ 24000 imágenes descomprimidas...
✅ 25000 imágenes descomprimidas...
✅ 26000 imágenes descomprimidas...
✅ 27000 imágenes descomprimidas...
✅ 28000 imágenes descomp

In [6]:
# ------------------- 5. Leer CSV y preparar rutas -------------------
df = pd.read_csv(csv_colab_path)

# Reemplazar '\' por '/' sin quitar carpeta
df['Ruta portada'] = df['Ruta portada'].apply(lambda x: str(x).replace('\\', '/') if pd.notnull(x) else '')

# Crear texto CLIP
textos_clip = df['Título'] + ' - ' + df['Autor/es']

# Crear ruta absoluta combinando extract_to con la ruta relativa completa
rutas_imagenes = df['Ruta portada'].apply(lambda x: os.path.join('/content', x) if x else '')

In [7]:
!pip install open-clip-torch ftfy --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.1 MB/s eta 0:00:00


In [8]:
import open_clip
import torch

# Cargar modelo y tokenizer
model, preprocess, tokenizer = open_clip.create_model_and_transforms(
    model_name="ViT-B-32",
    pretrained="laion2b_s34b_b79k"
)
tokenizer = open_clip.get_tokenizer("ViT-B-32")

# Asegurar uso de GPU si está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [9]:
from PIL import Image
import torch
from tqdm import tqdm

# ===================== Inicializar listas =====================
image_features_list = []
text_features_list = []
textos_ok = []
rutas_ok = []

# ===================== Procesar todo el dataset =====================
print(f"🔄 Procesando {len(textos_clip)} libros...")

for idx, (texto, ruta) in enumerate(tqdm(zip(textos_clip, rutas_imagenes), total=len(textos_clip))):
    try:
        # Cargar imagen
        img = Image.open(ruta).convert('RGB')
        img_tensor = preprocess(img).unsqueeze(0).to(device)

        # Tokenizar texto
        txt_tensor = tokenizer([texto]).to(device)

        # Obtener embeddings
        with torch.no_grad():
            img_feat = model.encode_image(img_tensor)
            txt_feat = model.encode_text(txt_tensor)

        # Normalizar
        img_feat /= img_feat.norm(dim=-1, keepdim=True)
        txt_feat /= txt_feat.norm(dim=-1, keepdim=True)

        # Guardar
        image_features_list.append(img_feat.cpu())
        text_features_list.append(txt_feat.cpu())
        textos_ok.append(texto)
        rutas_ok.append(ruta)

        # Mostrar cada 1000 procesados
        if idx > 0 and idx % 1000 == 0:
            print(f"✅ Procesadas {idx} imágenes correctamente")

    except Exception as e:
        print(f"⚠️ Error con imagen {idx}: {ruta} - {e}")

print("✅ Proceso completado. Generando archivo final...")


🔄 Procesando 50734 libros...


  2%|▏         | 1009/50734 [00:52<18:38, 44.47it/s]

✅ Procesadas 1000 imágenes correctamente


  4%|▍         | 2005/50734 [01:16<18:41, 43.46it/s]

✅ Procesadas 2000 imágenes correctamente


...

✅ Procesadas 49000 imágenes correctamente


 99%|█████████▊| 50009/50734 [20:30<00:15, 46.01it/s]

✅ Procesadas 50000 imágenes correctamente


100%|██████████| 50734/50734 [20:48<00:00, 40.65it/s]

⚠️ Error con imagen 50716: /content/portadas/La Casa de Hades - Rick Riordan.jpg - [Errno 2] No such file or directory: '/content/portadas/La Casa de Hades - Rick Riordan.jpg'
⚠️ Error con imagen 50717: /content/portadas/El laberinto de huesos - Rick Riordan.jpg - [Errno 2] No such file or directory: '/content/portadas/El laberinto de huesos - Rick Riordan.jpg'
⚠️ Error con imagen 50718: /content/portadas/La Marca de Atenea - Rick Riordan.jpg - [Errno 2] No such file or directory: '/content/portadas/La Marca de Atenea - Rick Riordan.jpg'
⚠️ Error con imagen 50719: /content/portadas/El hijo de Neptuno - Rick Riordan.jpg - [Errno 2] No such file or directory: '/content/portadas/El hijo de Neptuno - Rick Riordan.jpg'
⚠️ Error con imagen 50720: /content/portadas/El héroe perdido - Rick Riordan.jpg - [Errno 2] No such file or directory: '/content/portadas/El héroe perdido - Rick Riordan.jpg'
⚠️ Error con imagen 50721: /content/portadas/La sombra de la serpiente - Rick Riordan.jpg - [Errno 2

In [10]:
# Guardar en archivo .pkl
import pickle

# Guardar en archivo .pkl
datos_clip_full = {
    'image_features': torch.cat(image_features_list),
    'text_features': torch.cat(text_features_list),
    'textos': textos_ok,
    'rutas_imagenes': rutas_ok
}

with open('embeddings_clip_full.pkl', 'wb') as f:
    pickle.dump(datos_clip_full, f)

print("✅ Archivo embeddings_clip_full.pkl guardado con éxito.")


✅ Archivo embeddings_clip_full.pkl guardado con éxito.


In [11]:
from google.colab import files
files.download('embeddings_clip_full.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>